<a href="https://colab.research.google.com/github/avena-co/airbyte/blob/master/table_chair_test_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
# THE DATASET
# A set of N number of samples
# Each sample consists of a table and a chair
# where the chair is translated 1 unit in the
# reverse direction of the table, simulating a
# person using it
# Both table and chair is made up of 2 unit vectors,
# representing the position and direction of the item
# in 2D space

import torch
import math

torch.manual_seed(13)

def random_unit_vector():
  vector = torch.rand(2)
  length = math.sqrt(vector[0]**2+vector[1]**2)
  vector[0] /= length
  vector[1] /= length
  return vector

def generate_dataset(N):
  X = torch.zeros([N, 8])
  y = torch.zeros([N, 8])

  for i in range(N):
    table_xy = random_unit_vector()
    table_uv = random_unit_vector()
    chair_xy = table_xy - table_uv
    chair_uv = table_uv
    X[i] = torch.cat([table_xy, table_uv, chair_xy, chair_uv])
    y[i] = torch.cat([table_xy, table_uv, torch.zeros_like(chair_xy), torch.zeros_like(chair_uv)])

  return X, y

In [159]:
X, y = generate_dataset(10000)
train_split = int(0.8 * len(X))
X_train = X[:train_split]
X_test = X[train_split:]
y_train = X[:train_split]
y_test = y[train_split:]

In [160]:
# THE MODEL

from torch import nn

class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Linear(8, 8)

  def forward(self, x):
    return self.model.forward(x)

class TableChairModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
      nn.Linear(8, 4),
    )

    self.decoder = nn.Sequential(
          nn.Linear(4, 8),
    )

  def forward(self, x):
      encoded = self.encoder(x)
      decoded = self.decoder(encoded)
      return decoded

In [161]:
# TRAINING LOOP

def train(model, X_train, y_train, X_test, y_test, ages, epochs):
  for age in range(ages):
    loss_fn = torch.nn.L1Loss()
    optimizer = torch.optim.SGD(params=model.parameters(), lr=10**-age)
    for epoch in range(epochs):
      model.train()
      y_preds = model(X_train)
      loss = loss_fn(y_preds, y_train)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if epoch % 4 == 0:
        model.eval()
        with torch.inference_mode():
          y_preds = model(X_test)
          loss = loss_fn(y_preds, y_test)
          print(f"Age: {age} | Epoch: {epoch + 1} | Loss: {loss:.5f}")

In [162]:
# model = TableChairModel()
model = LinearRegressionModel()

In [163]:
train(model, X_train, y_train, X_test, y_test, 10, 1000)

Age: 0 | Epoch: 1 | Loss: 0.39481
Age: 0 | Epoch: 5 | Loss: 0.29014
Age: 0 | Epoch: 9 | Loss: 0.30077
Age: 0 | Epoch: 13 | Loss: 0.33525
Age: 0 | Epoch: 17 | Loss: 0.35243
Age: 0 | Epoch: 21 | Loss: 0.35267
Age: 0 | Epoch: 25 | Loss: 0.35280
Age: 0 | Epoch: 29 | Loss: 0.35284
Age: 0 | Epoch: 33 | Loss: 0.35286
Age: 0 | Epoch: 37 | Loss: 0.35284
Age: 0 | Epoch: 41 | Loss: 0.35279
Age: 0 | Epoch: 45 | Loss: 0.35274
Age: 0 | Epoch: 49 | Loss: 0.35272
Age: 0 | Epoch: 53 | Loss: 0.35270
Age: 0 | Epoch: 57 | Loss: 0.35267
Age: 0 | Epoch: 61 | Loss: 0.35265
Age: 0 | Epoch: 65 | Loss: 0.35263
Age: 0 | Epoch: 69 | Loss: 0.35260
Age: 0 | Epoch: 73 | Loss: 0.35258
Age: 0 | Epoch: 77 | Loss: 0.35256
Age: 0 | Epoch: 81 | Loss: 0.35253
Age: 0 | Epoch: 85 | Loss: 0.35253
Age: 0 | Epoch: 89 | Loss: 0.35253
Age: 0 | Epoch: 93 | Loss: 0.35254
Age: 0 | Epoch: 97 | Loss: 0.35254
Age: 0 | Epoch: 101 | Loss: 0.35254
Age: 0 | Epoch: 105 | Loss: 0.35254
Age: 0 | Epoch: 109 | Loss: 0.35253
Age: 0 | Epoch: 113 

In [164]:
# INFERENCE

model.eval()
with torch.inference_mode():
  y_preds = model(X_test)
print(X_test[0])
print(y_preds[0])

tensor([ 0.9871,  0.1600,  0.8083,  0.5888,  0.1789, -0.4288,  0.8083,  0.5888])
tensor([ 0.9908,  0.1643,  0.8109,  0.5886,  0.1792, -0.4307,  0.8079,  0.5918])
